In [132]:
from scipy import sparse, io
import pickle
import numpy as np

In [133]:
discountFactor = 0.9925 
numActions = 4
numStates = 81
rewards = (np.loadtxt("rewards.txt")).astype(int)
west = np.loadtxt("prob_a1.txt").astype(float)
north = np.loadtxt("prob_a2.txt").astype(float)
east = np.loadtxt("prob_a3.txt").astype(float)
south = np.loadtxt("prob_a4.txt").astype(float)
#Transition matrices
westMat = sparse.csr_matrix((west[:,2], (west[:,0].astype(int)-1, west[:,1].astype(int)-1)), shape=(numStates, numStates)).toarray()
northMat = sparse.csr_matrix((north[:,2], (north[:,0].astype(int)-1, north[:,1].astype(int)-1)), shape=(numStates, numStates)).toarray()
eastMat = sparse.csr_matrix((east[:,2], (east[:,0].astype(int)-1, east[:,1].astype(int)-1)), shape=(numStates, numStates)).toarray()
southMat = sparse.csr_matrix((south[:,2], (south[:,0].astype(int)-1, south[:,1].astype(int)-1)), shape=(numStates, numStates)).toarray()
transMat = [westMat, northMat, eastMat, southMat]
del west, east, north, south
policy = np.random.randint(0, 3, size=numStates) #setting initial policy randomly
valueMat = np.zeros(numStates).astype(int) #V^pi
greedyPolicy = np.zeros(numStates).astype(int) #pi^*

In [134]:
#updateValueMat
def updateValueMat():
    global valueMat
    probMat = np.zeros(shape=(numStates, numStates))
    for i, s in enumerate(policy):
        probMat[i,:] = transMat[s][i,:]
    probMat = np.identity(numStates) - discountFactor*probMat
    probMat = np.linalg.inv(probMat)
    valueMat = np.matmul(probMat, rewards)

In [135]:
def updateGreedyPolicy():
    global greedyPolicy, valueMat
    for s1 in range(numStates):
        maxSum = float('-inf')
        for a in range(numActions):
            summ = 0;
            for s2 in range(numStates):
                summ += (transMat[a][s1][s2]*valueMat[s2])
            if(summ>maxSum):
                maxSum = summ
                greedyPolicy[s1] = a

In [108]:
oldValueMat = valueMat
print(greedyPolicy)
while(True):
    updateValueMat()
    print(valueMat)
    updateGreedyPolicy()
    print(greedyPolicy)
    if(np.array_equal(valueMat, oldValueMat)):
        print("Convergence reached")
        print(valueMat)
        break
    policy = greedyPolicy
    valueMat = oldValueMat

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.           -1.75696306    0.            0.            0.
    0.            0.            0.            0.           -2.32108014
   -2.28803502    0.            0.           -1.80592663   -1.53399108
   -1.7257304     0.            0.           -2.34093503    0.           -3.7035939
   -2.40268546   -1.82802954    0.           -9.34171691    0.            0.
   -2.36059564   -3.80156811   -3.76900965    0.            0.
   -9.71011721   -9.6238704     0.            0.            0.
  -58.29919677    0.            0.            0.          -12.73141205
    0.            0.            0.         -133.33333333 -130.41874888
 -133.33333333    0.         -133.33333333 -128.36760665  -97.75775558
    0.            0.         -132.25374614 -131.20537619 -131.8130013
 -114.49073012 -131.77522875 -129.68305675  -

[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.          100.70098073    0.            0.            0.
    0.            0.            0.            0.          102.3752644
  101.52364515    0.            0.          109.48993454  110.40903296
  111.33584663    0.            0.          103.23462342    0.
  106.77826755  107.67462643  108.57848712    0.          112.27044032
    0.            0.          104.10121204  104.97507555  105.88853591
    0.            0.          114.1632295   113.21287932    0.            0.
    0.          103.78140737    0.            0.            0.
  115.12155727    0.            0.            0.         -133.33333333
   90.9853796  -133.33333333    0.         -133.33333333  116.08792959
  122.02491241    0.            0.           81.39949278   93.67165583
   95.17285726  108.34261934  109.58365072  123.64307021  

[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.          100.70098073    0.            0.            0.
    0.            0.            0.            0.          102.3752644
  101.52364515    0.            0.          109.48993454  110.40903296
  111.33584663    0.            0.          103.23462342    0.
  106.77826755  107.67462643  108.57848712    0.          112.27044032
    0.            0.          104.10121204  104.97507555  105.88853591
    0.            0.          114.1632295   113.21287932    0.            0.
    0.          103.78140737    0.            0.            0.
  115.12155727    0.            0.            0.         -133.33333333
   90.9853796  -133.33333333    0.         -133.33333333  116.08792959
  122.02491241    0.            0.           81.39949278   93.67165583
   95.17285726  108.34261934  109.58365072  123.64307021  

[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.          100.70098073    0.            0.            0.
    0.            0.            0.            0.          102.3752644
  101.52364515    0.            0.          109.48993454  110.40903296
  111.33584663    0.            0.          103.23462342    0.
  106.77826755  107.67462643  108.57848712    0.          112.27044032
    0.            0.          104.10121204  104.97507555  105.88853591
    0.            0.          114.1632295   113.21287932    0.            0.
    0.          103.78140737    0.            0.            0.
  115.12155727    0.            0.            0.         -133.33333333
   90.9853796  -133.33333333    0.         -133.33333333  116.08792959
  122.02491241    0.            0.           81.39949278   93.67165583
   95.17285726  108.34261934  109.58365072  123.64307021  

    0.            0.        ]
[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.          100.70098073    0.            0.            0.
    0.            0.            0.            0.          102.3752644
  101.52364515    0.            0.          109.48993454  110.40903296
  111.33584663    0.            0.          103.23462342    0.
  106.77826755  107.67462643  108.57848712    0.          112.27044032
    0.            0.          104.10121204  104.97507555  105.88853591
    0.            0.          114.1632295   113.21287932    0.            0.
    0.          103.78140737    0.            0.            0.
  115.12155727    0.            0.            0.         -133.33333333
   90.9853796  -133.33333333    0.         -133.33333333  116.08792959
  122.02491241    0.            0.           81.39949278   93.67165583
   95.17285726  108.34261934

...

[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.          100.70098073    0.            0.            0.
    0.            0.            0.            0.          102.3752644
  101.52364515    0.            0.          109.48993454  110.40903296
  111.33584663    0.            0.          103.23462342    0.
  106.77826755  107.67462643  108.57848712    0.          112.27044032
    0.            0.          104.10121204  104.97507555  105.88853591
    0.            0.          114.1632295   113.21287932    0.            0.
    0.          103.78140737    0.            0.            0.
  115.12155727    0.            0.            0.         -133.33333333
   90.9853796  -133.33333333    0.         -133.33333333  116.08792959
  122.02491241    0.            0.           81.39949278   93.67165583
   95.17285726  108.34261934  109.58365072  123.64307021  

[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.          100.70098073    0.            0.            0.
    0.            0.            0.            0.          102.3752644
  101.52364515    0.            0.          109.48993454  110.40903296
  111.33584663    0.            0.          103.23462342    0.
  106.77826755  107.67462643  108.57848712    0.          112.27044032
    0.            0.          104.10121204  104.97507555  105.88853591
    0.            0.          114.1632295   113.21287932    0.            0.
    0.          103.78140737    0.            0.            0.
  115.12155727    0.            0.            0.         -133.33333333
   90.9853796  -133.33333333    0.         -133.33333333  116.08792959
  122.02491241    0.            0.           81.39949278   93.67165583
   95.17285726  108.34261934  109.58365072  123.64307021  

[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.          100.70098073    0.            0.            0.
    0.            0.            0.            0.          102.3752644
  101.52364515    0.            0.          109.48993454  110.40903296
  111.33584663    0.            0.          103.23462342    0.
  106.77826755  107.67462643  108.57848712    0.          112.27044032
    0.            0.          104.10121204  104.97507555  105.88853591
    0.            0.          114.1632295   113.21287932    0.            0.
    0.          103.78140737    0.            0.            0.
  115.12155727    0.            0.            0.         -133.33333333
   90.9853796  -133.33333333    0.         -133.33333333  116.08792959
  122.02491241    0.            0.           81.39949278   93.67165583
   95.17285726  108.34261934  109.58365072  123.64307021  

    0.            0.        ]
[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]
[   0.            0.          100.70098073    0.            0.            0.
    0.            0.            0.            0.          102.3752644
  101.52364515    0.            0.          109.48993454  110.40903296
  111.33584663    0.            0.          103.23462342    0.
  106.77826755  107.67462643  108.57848712    0.          112.27044032
    0.            0.          104.10121204  104.97507555  105.88853591
    0.            0.          114.1632295   113.21287932    0.            0.
    0.          103.78140737    0.            0.            0.
  115.12155727    0.            0.            0.         -133.33333333
   90.9853796  -133.33333333    0.         -133.33333333  116.08792959
  122.02491241    0.            0.           81.39949278   93.67165583
   95.17285726  108.34261934

KeyboardInterrupt: 

In [112]:
print([i+1 for (i,x) in enumerate(valueMat) if x>0]) # positive
print([i+1 for (i,x) in enumerate(valueMat) if x<0])
print(greedyPolicy)

[3, 11, 12, 15, 16, 17, 20, 22, 23, 24, 26, 29, 30, 31, 34, 35, 39, 43, 48, 52, 53, 56, 57, 58, 59, 60, 61, 62, 66, 70, 71, 79]
[47, 49, 51, 65, 67, 69]
[0 0 2 0 0 0 0 0 0 0 2 1 0 0 3 3 2 0 0 2 0 3 3 0 0 2 0 0 3 3 0 0 0 2 1 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 3 3 2 2 0 0 0 0 0 0 0 2 1 0 0 0
 0 0 0 0 0 0 0]


In [115]:
l = []
for j in [i for (i,x) in enumerate(valueMat) if x>0]:
    l.append(greedyPolicy[j])
print(l)

[2, 2, 1, 3, 3, 2, 2, 3, 3, 0, 2, 3, 3, 0, 2, 1, 0, 2, 0, 2, 2, 3, 3, 3, 3, 3, 2, 2, 0, 2, 1, 0]


In [147]:
#Value Iteration
valueMat = np.zeros(numStates) #V^pi
oldValueMat = np.zeros(numStates)
greedyPolicy = np.zeros(numStates).astype(int) #pi^*

In [148]:
def updateValueMat2():
    global valueMat, oldValueMat
    for s in range(numStates):
        valueMat[s] = rewards[s]
        print(rewards[s])
        maxSum = float('-inf')
        for a in range(numActions):
            summ = 0.0
            for s1 in range(numStates):
                summ += (transMat[a][s][s1]*oldValueMat[s1])
            if(summ>maxSum):
                maxSum = summ
                greedyPolicy[s] = a
        valueMat[s] += discountFactor*maxSum
    print(valueMat)

In [149]:
while(True):
    updateValueMat2()
    #print(oldValueMat)
    #print(valueMat)
    if(np.array_equal(oldValueMat, valueMat)):
        print("Here")
        break
    oldValueMat = valueMat
    print(valueMat)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
-1
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
-1
0
-1
0
0
0
0
0
0
0
0
0
1
0
0
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
-1
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
-1
0
-1
0
0
0
0
0
0
0
0
0
1
0
0
[  0

In [143]:
print([i+1 for (i,x) in enumerate(valueMat) if x>0])

[3, 11, 12, 15, 16, 17, 20, 22, 23, 24, 26, 29, 30, 31, 34, 35, 39, 43, 48, 52, 53, 56, 57, 58, 59, 60, 61, 62, 66, 71, 79]


In [144]:
l = []
for j in [i for (i,x) in enumerate(valueMat) if x>0]:
    l.append(greedyPolicy[j])
print(l)

[2, 2, 1, 3, 3, 2, 2, 3, 3, 0, 2, 3, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 1, 0, 1, 0]
